In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage
import requests, json, os
from dotenv import load_dotenv
load_dotenv()


True

In [36]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.8)
user_request = """please identify the top stock movers today, look for the news that moved these stocks, and summarize the news to explain why each of these stock moved. 
Please follow this format: company name, percentage of price change and explanation
"""

functions = [
    {
        "name":"get_stock_movers",
        "description": "Get the stocks that has biggest price/volume moves, e.g. actives, gainers, losers, etc.",
        "parameters": {
            "type": "object",
            "properties": {

            }
        }
    },
    {
        "name": "get_market_news",
        "descriptions":"Get the latest news that moves the stocks",
        "parameters": {
            "type": "object",
            "properties": {
                "performanceIDs": {
                    "type": "array",
                    "items": {
                        "type":"string",
                        "description": "a performanceID in a specific format such as '0P000003B1' of a stock"
                    },
                    "description": "a list of all performanceIDs"
                }
            },
            "required": ["performanceIDs"]
        }
    }
]

In [27]:
#helper function
def get_stock_movers():
    url = "https://morning-star.p.rapidapi.com/market/v2/get-movers"
    headers = {
        "X-RapidAPI-Key": os.getenv("X-RAPIDAPI-KEY"),
        "X-RapidAPI-Host": "morning-star.p.rapidapi.com"
    }
    response = requests.get(url, headers=headers)
    data = response.json()
    movers = []
    for i in data["gainers"][:3]:
        movers.append({i["ticker"], i["name"], i["performanceID"], i["percentNetChange"]})
    return movers

def get_market_news(performanceID):
    url = "https://morning-star.p.rapidapi.com/news/list"

    querystring = {"performanceId":performanceID}

    headers = {
        "X-RapidAPI-Key": os.getenv("X-RAPIDAPI-KEY"),
        "X-RapidAPI-Host": "morning-star.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
  
    return response.json()

def format_news(list_ids):
    list_news = []
    for i in list_ids:
        list_title = []    
        list_news.append({i: list_title})
        for j in get_market_news(i)[:3]:
            list_title.append(j["title"])
    return list_news

In [37]:
first_response = llm.predict_messages([HumanMessage(content=user_request)], functions=functions, function_call="auto")
api_response = first_response.additional_kwargs["function_call"]["arguments"]
api_response = str(get_stock_movers())

In [38]:
print(api_response)

[{'LRN', 'Stride', '0P0000A298', 17.9562}, {'0P0000HX9P', 9.7787, 'JBT', 'John Bean Technologies Corp'}, {8.9361, 'MSA', '0P000003NM', 'Mine Safety'}]


In [41]:
second_response = llm.predict_messages([HumanMessage(content=user_request),
                                       AIMessage(content=api_response)], functions=functions, function_call="auto")

In [42]:
print(second_response)

content='' additional_kwargs={'function_call': {'name': 'get_market_news', 'arguments': '{\n  "performanceIDs": ["LRN", "0P0000A298", "0P0000HX9P", "MSA", "0P000003NM"]\n}'}}


In [43]:
list_ids = json.loads(second_response.additional_kwargs["function_call"]["arguments"])
list_ids = list_ids["performanceIDs"]
second_api_response= str(format_news(list_ids))

In [47]:
print(type(second_api_response))
print(second_api_response)

<class 'str'>
[{'LRN': ['Stride Professional Development Center Announces 2024 Promising Practices Conference', 'Stocks to Watch: Microsoft, Alphabet, Stride', 'Stride Achieves Record Q1 Revenue']}, {'0P0000A298': ['Stride Professional Development Center Announces 2024 Promising Practices Conference', 'Stocks to Watch: Microsoft, Alphabet, Stride', 'Stride Achieves Record Q1 Revenue']}, {'0P0000HX9P': ['JBT Corporation Reports Third Quarter 2023 Results', 'JBT Corporation Announces Third Quarter 2023 Earnings Release and Conference Call Schedule', 'JBT Corporation Declares Quarterly Dividend']}, {'MSA': ['MSA Safety Announces Third Quarter Results', 'MSA Safety Highlights Enhanced Connected Safety Solutions at National Safety Congress & Expo', 'MSA Safety Schedules Third Quarter 2023 Earnings Conference Call']}, {'0P000003NM': ['MSA Safety Announces Third Quarter Results', 'MSA Safety Highlights Enhanced Connected Safety Solutions at National Safety Congress & Expo', 'MSA Safety Schedu

In [48]:
final_response = llm.predict_messages([HumanMessage(content=user_request),
                                       AIMessage(content=api_response),
                                       AIMessage(content=second_api_response)],functions=functions, function_call="auto")


In [50]:
print(final_response.content)

The top stock movers today are:

1. Stride (LRN): +17.96%
    - Stride Professional Development Center announced the 2024 Promising Practices Conference.
    - Microsoft, Alphabet, and Stride are stocks to watch.
    - Stride achieved record Q1 revenue.

2. John Bean Technologies Corp (JBT): +9.78%
    - JBT Corporation reported third quarter 2023 results.
    - JBT Corporation announced the third quarter 2023 earnings release and conference call schedule.
    - JBT Corporation declared a quarterly dividend.

3. Mine Safety (MSA): +8.94%
    - MSA Safety announced third quarter results.
    - MSA Safety highlighted enhanced connected safety solutions at the National Safety Congress & Expo.
    - MSA Safety scheduled the third quarter 2023 earnings conference call.
